In [22]:
import yaml
import pandas as pd

folder_location = 'C:/Users/ttrol/CodingProjects/MatNet/scripts/webscraper/results_files/'
filename = 'AISI_steels.yaml'
file_location = folder_location + filename

with open(file_location, "r") as stream:
    try:
        raw_materials = yaml.safe_load(stream)
        print('Raw Size: ', len(raw_materials))
        raw_df = pd.DataFrame(raw_materials)
    except yaml.YAMLError as exc:
        print(exc)

Raw Size:  1813


In [23]:
import time

def time_it(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Function '{func.__name__}' executed in {execution_time:.6f} seconds.")
        return result
    return wrapper

In [24]:
import requests
import random
from bs4 import BeautifulSoup

#oxylabs authentication
USERNAME = "pcho69"
PASSWORD = "StealthyWebsitePumpk1n"
ENDPOINT = "pr.oxylabs.io:7777"


entry = ('http://customer-%s:%s@%s' %
    (USERNAME, PASSWORD, ENDPOINT))
query = {
    'http': entry,
    'https': entry,
}
session = requests.Session()


# link = raw_df.link[random.randint(0,len(raw_df)-1)]
link = raw_df.link[0]

# for i in range(1):
# @time_it
def parse_page(link, sesh):
    result = {}
    result['source'] = link

    response = sesh.get(
        headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
        },
        url=link,
        proxies= query,
        verify=True
    )

    soup = BeautifulSoup(response.content,'html')

    #find name of material in page and remove unnecessary whitespace
    material_name = soup.find('title').get_text() # pyright: ignore[reportOptionalMemberAccess]  # noqa: E501
    material_name = material_name.strip()
    result['name'] = material_name

    table = soup.select('div#ctl00_ContentMain_ucDataSheet1_pnlMaterialData table.tabledataformat')
    if len(table) != 2:
        raise Exception('IP Blocked')

    #find supplementary notes in tables
    matl_notes_table = table[0].find(id= "ctl00_ContentMain_ucDataSheet1_trMatlNotes")
    category_table = table[0].find(id= "ctl00_ContentMain_ucDataSheet1_trMatlGroups")
    
    material_notes = None
    categories = None

    try:
        if matl_notes_table is not None:
            material_notes = matl_notes_table.find('td').text   #type: ignore
        
        if category_table is not None:
            categories = category_table.find('td').text  #type: ignore
            categories = [item.strip() for item in categories.split(";")]
        
        result['categories'] = categories
        result['material_notes'] = material_notes
    except Exception:
        exception_msg = 'Issues with grabbing material and category notes: ' \
            + link
        print(exception_msg)
        

    #Second table in html page has correct content
    main_table_rows = table[1].find_all('tr')
    property_tables = []
    if main_table_rows is None:
        raise TypeError('Cannot parse None type')
  
    
    #This creates a list of tables from the main content table. 
    property_type = None
    for row in main_table_rows:
        type = row.select('th:not([class])')
        if len(type) > 0:
            property_type = type[0].text.strip()
            property_type = property_type.replace(',','').replace(' ','_').lower()
            continue

        prop = row.select('td:not([class])')
        if len(prop) > 0:
            prop = prop[0].text.strip().replace(',','').replace(' ','_').lower()
        if prop:
            ele = row.select('td.dataCell')[0]
            if ele == []:
                raise Exception('Cannot find dataCell')
            
            if property_type not in result:
                result[property_type] = {}

            if prop not in result[property_type]:
                result[property_type][prop] = ele.text.strip()

    return result

# result = parse_page(link,session)
# result


In [25]:
from pymongo import MongoClient
from tqdm.notebook import tqdm

i = 0

#connect to mongodb client
client = MongoClient("mongodb://localhost:27017/")
db = client.matjet
collection = db.materials


session = requests.Session()
link_range = range(398,len(raw_df.link))

n = 4
for i in tqdm(link_range, desc="Processing", unit="iteration"):
    if (i + 1) % n == 0:
        session = requests.Session()
    
    try:
        document = parse_page(link = raw_df.link[i], sesh=session)
    except Exception:
        raise Exception(f'Error at index {i}')
    
    duplicates = collection.count_documents({'name': document['name']})    
    
    if duplicates > 0:
        print('Duplicate found: Did not add.')
    else:
        collection.insert_one(document)
    

Processing:   0%|          | 0/1415 [00:00<?, ?iteration/s]